# ¿Cuánto deberían valer las propiedades en Milwaukee, Wisconsin? (Parte II)

## Objetivos

En los últimos casos, hemos visto varias técnicas utilizadas para diagnosticar la aplicabilidad de los modelos de regresión lineal.

## Introducción 

**Problema empresarial.** Su tarea es **crear un modelo para predecir los precios de las propiedades en la ciudad de Milwaukee, Wisconsin**.

**Contexto analítico.**  el conjunto de datos consta de ventas de propiedades (comerciales y residenciales) en Milwaukee, Wisconsin, de 2002 a 2018. 


# Preparar data

In [1]:
### Load relevant packages
import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import seaborn                 as sns
import statsmodels.formula.api as smf
import statsmodels.api         as sm
import scipy

%matplotlib inline
plt.style.use('ggplot')
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
data = pd.read_csv("2002-2018-property-sales-data.csv",
    dtype = { # Categorias
        "PropType": "category",
        "District": "category",
        "Extwall": "category",
        "Nbhd": "category",
        "Style": "category",
    },
    parse_dates=["Sale_date"],
)
def remove_unused_categories(data):
    """ The `remove_unused_categories` method in pandas
        removes categories from a Series if there are no
        elements of that category.
        
        This function is a convenience function that removes
        unused categories for all categorical columns
        of a data frame.
        
        The reason this is useful is that when we
        fit a linear regression, `statsmodels` will
        create a coefficient for every category in a column,
        and so unused categories pollute the results.
    """
    for cname in data:
        col = data[cname]
        if pd.api.types.is_categorical_dtype(col):
            data[cname] = col.cat.remove_unused_categories()
    return data

clean = np.where(
    (data["Sale_price"] > 2000) & # Importante
    (data["Year_Built"] > 1800) &
    (data["Fin_sqft"] > 0) & # Condicion logica
    (data["Lotsize"] > 0)  & # Condicion Logica
    (data["PropType"] == "Residential")
    )
data_clean = data.iloc[clean].copy()
remove_unused_categories(data_clean).head()

/var/folders/wt/dd5nh3v12zvd92jwl2f9nt400000gn/T/ipykernel_79723/1550737232.py:27: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(col):


,PropType,Taxkey,Address,CondoProject,District,Nbhd,Style,Extwall,Stories,Year_Built,Nr_of_rms,Fin_sqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
10,Residential,3080013000,3033 N 35TH ST,NaN,7,2960,AP 1,Frame,2.0,1913,0,3476,4,9,1,0,5040,2002-02-01,42000
51,Residential,3190434000,1908 E WEBSTER PL,NaN,3,3170,Rm or Rooming House,Frame,2.0,1897,0,1992,4,2,2,0,2880,2002-05-01,145000
67,Residential,3891722000,812 N 25TH ST,NaN,4,3040,Rm or Rooming House,Frame,2.0,1907,0,2339,6,0,1,0,3185,2002-06-01,30000
116,Residential,3880628000,959 N 34TH ST,NaN,4,2300,AP 1,Frame,2.0,1890,0,2329,4,4,1,0,5781,2002-10-01,66500
134,Residential,3880406000,3209 W WELLS ST,NaN,4,2300,Mansion,Stone,2.5,1891,0,7450,2,7,6,0,15600,2002-11-01,150500


Debemos dividir aleatoriamente los datos en un conjunto de entrenamiento y un conjunto de prueba. El **conjunto de entrenamiento** es aquel en el que entrenamos y ajustamos nuestro modelo de regresión lineal múltiple. Luego ejecutamos nuestro modelo ajustado en el **conjunto de prueba** y comparamos sus predicciones con los datos de la variable de respuesta del conjunto de prueba real para evaluar su rendimiento.

In [ ]:
#### RESPUESTA AQUI

Ahora apodemos hacer un modelo para poder predecir `Sales` en funcion de `District + Units+Fin_sqft`

Analiza los resultados , hacen sentido?

In [ ]:
#### RESPUESTA AQUI

Ahora intentemos realizar algunas transformaciones para entender si se mejoran nuestros resultados
Puedes usar esto como referencia para hacer el modelo 
```python
model_log = smf.ols(formula = "np.log(Sale_price) ~ District + Units"
                           "+ np.log(Fin_sqft)", 
                 data = train).fit()
```

In [ ]:
#### RESPUESTA AQUI

# Variables categoricas

Quizás haya notado que hay más de una docena de coeficientes para `Distrito` arriba. Esto se debe a que `Distrito` es una variable categórica y, para las características categóricas, se obtiene un coeficiente para todas las categorías excepto una. Si solo hay dos categorías (por ejemplo, género), esto es intuitivo: la característica se convierte en una columna de ceros y unos antes de introducirla en la regresión, donde se trata como una variable numérica regular.

Cuando hay más de dos categorías, una categoría se designa como la categoría de "referencia" o "línea de base", y se crean columnas "ficticias" de unos y ceros para todas las demás categorías. Tomemos un ejemplo ficticio con tres categorías y cinco filas:

| Categoría |
|--------------|
| A |
| B |
| C |
| C |
| A |

Antes de ajustar la regresión lineal, la columna `Categoría` se transforma en **dos** columnas “ficticias” (no se agrega una columna para la categoría de referencia).

La primera columna es 1 si el distrito es `B` y 0 en caso contrario, mientras que la segunda columna es 1 si el distrito es `C` y 0 en caso contrario. Obtenemos:

| Categoría_B | Categoría_C |
|--------------|-------------|
| 0 | 0 |
| 1 | 0 |
| 0 | 1 |
| 0 | 1 |
| 0 | 0 |

Las columnas ficticias se introducen en la regresión lineal y se tratan como variables numéricas regulares. Esta técnica se denomina **codificación one-hot** y se puede realizar manualmente con la función pandas `pd.get_dummies()`:

In [3]:
df_dummy=pd.DataFrame({'Category':['A','B','C','C','A']})
pd.get_dummies(df_dummy,columns=['Category'], drop_first=True) 

,Category_B,Category_C
0,False,False
1,True,False
2,False,True
3,False,True
4,False,False


Al igual que en el caso binario, la elección de la línea de base cambia los coeficientes y su interpretación; el coeficiente `Distrito[T.3]` de 1.0236 debe interpretarse como la diferencia en los resultados previstos entre los distritos 3 y 1. Pero la elección de la línea de base no afecta las predicciones ni el rendimiento del modelo, por lo que la mayoría del software elegirá arbitrariamente una categoría como línea de base sin previo aviso (a menudo la primera alfabéticamente).

Ahora hay un nuevo desafio 

Agrega la columna `Style` a nuestro último modelo de regresión. Hazlo de dos maneras: primero transformando la variable `Style` usando `pd.get_dummies()` y luego sin la transformación. Verifica que obtengas los mismos resultados. De acuerdo con este modelo, ¿qué propiedad style es la más deseable?

**Sugerencia:** La variable `Style` contiene nombres, como `Residence O/S`, que pueden ser problemáticos al escribir fórmulas `smf.ols` (consulta la [Sintaxis de Pasty](https://patsy.readthedocs.io/en/latest/index.html)). Puedes usar la sintaxis `Pasty` entre comillas `Q()` en los nombres de variables para evitar este problema:
```python
formula="np.log(Sale_price) ~Q("Residence O/S")+...
```

In [ ]:
#### RESPUESTA AQUI